In [1]:
### Packages

using Printf
using LinearAlgebra
using JLD
using QuadGK
using Statistics
using PyCall
using PyPlot
using LaTeXStrings
using GadgetIO
using GadgetUnits
using GadgetGalaxies
using Unitful
using UnitfulAstro
using Missings
using Distributions
using CSV
using DataFrames
using Cosmology
using Suppressor

println("Done!")


Done!


In [2]:
### Settings

box         = "/HydroSims/Magneticum/Box4/uhr_test"
input_dir   = "/home/moon/sfortune/spinevo/halostories_v20211007_min0.0Gyr"
output_dir  = "/home/moon/sfortune/spinevo/mergerimpact"
mass_ST_thr = 1e10

storyfilelist   = readdir(input_dir)
print("done")


done

In [3]:
assembly_DM     = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_DM")
println("Done 1/3")
assembly_GAS    = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_GAS")
println("Done 2/3")
assembly_STARS  = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_STARS")
println("Done 3/3")


Dict{String, Array} with 32 entries:
  "I_SUB"         => Union{Missing, Int64}[0, 0, 0, 466, 671, 656, 715, 719, 73…
  "M_MISSED"      => Union{Missing, Float64}[missing, missing, missing, missing…
  "Merger_Map"    => Union{Missing, Float64}[1.61713e10 3.59224e9 … 2.79697e8 2…
  "M_fromJ"       => Union{Missing, Float64}[7.75752e11, 9.81942e11, 1.08073e12…
  "SNAP"          => Union{Missing, Int64}[40, 44, 48, 52, 58, 60, 64, 68, 72, …
  "M2_MISSED"     => Union{Missing, Float64}[missing, missing, missing, missing…
  "δM2_felix"     => Union{Missing, Float64}[5.69021e11, 4.78446e11, 1.44396e11…
  "M2_CONSIDERED" => Union{Missing, Float64}[2.01757e10, 3.51743e11, 6.82604e10…
  "REDSHIFT"      => Union{Missing, Float64}[1.70635, 1.47758, 1.32344, 1.17889…
  "M2_felix"      => Union{Missing, Float64}[5.69021e11, 1.04747e12, 1.19186e12…
  "ϕ_flip"        => Union{Missing, Float64}[10.5283, 155.146, 23.5839, 145.292…
  "M_MM"          => Union{Missing, Float64}[1.61713e10, 1.0348e11, 1.67

In [4]:
#array of max merger indices
main_mm_indices = missings(Int64, 0)
MMdm_indices    = missings(Int64, 0)
for i in 1:length(assembly_DM["M2_felix"])
    if sum( assembly_DM["Merger_Map"][2, 1+sum(assembly_DM["N_MERGERS"][1:i-1]) : sum(assembly_DM["N_MERGERS"][1:i])] ) > 0. # is there a merger?
        global MMdm_indices    = vcat( MMdm_indices, sum(assembly_DM["N_MERGERS"][1:i-1]) + argmax( assembly_DM["Merger_Map"][2,1+sum(assembly_DM["N_MERGERS"][1:i-1]):sum(assembly_DM["N_MERGERS"][1:i])] ) )
        global main_mm_indices = vcat( main_mm_indices, i)
    end
end
m2_felix    = assembly_DM["Merger_Map"][4,MMdm_indices]
m2_mm       = assembly_DM["Merger_Map"][2,MMdm_indices]
println("$(size(m2_felix))   $(size(m2_mm))")
println(length(MMdm_indices))
println( length(MMdm_indices)-count(ismissing, MMdm_indices) )
println(count(ismissing, log10.(m2_felix ./ m2_mm) ) )
println(count(ismissing, replace( log10.(m2_felix ./ m2_mm), missing=>NaN ) ) )
println(count(isnan, replace( log10.(m2_felix ./ m2_mm), missing=>NaN ) ) )
println(count(ismissing, replace( replace( log10.(m2_felix ./ m2_mm), missing=>NaN ), Inf=>NaN ) ) )
println(count(isnan, replace( replace( log10.(m2_felix ./ m2_mm), missing=>NaN ), Inf=>NaN ) ) )
println(size(assembly_STARS["ϕ_flip"][main_mm_indices]))


(2278,)   (2278,)
2278
2278


0
0
0
0
0


(2278,)


In [10]:
#count(isnan,assembly_DM["Merger_Map"][4,MMdm_indices] ./ assembly_DM["Merger_Map"][2,MMdm_indices])
count(iszero,assembly_DM["Merger_Map"][2,MMdm_indices])

0

In [48]:
bin_factor  = 20
ϕmin        = 0#minimum(assembly_STARS["ϕ_flip"])
ϕmax        = 180#maximum(assembly_STARS["ϕ_flip"])
ymin        = 0.1 #-2 #minimum(assembly_DM["Merger_Map"][4,MMdm_indices] ./ assembly_DM["Merger_Map"][2,MMdm_indices])
ymax        = 20 #6 #maximum(assembly_DM["Merger_Map"][4,MMdm_indices] ./ assembly_DM["Merger_Map"][2,MMdm_indices])
matplotlib  = pyimport("matplotlib")
axes_grid1  = pyimport("mpl_toolkits.axes_grid1")
pltcolors   = pyimport("matplotlib.colors")
matplotlib.use("qt5agg")




fig, ax = subplots()

h = ax.hexbin( replace(assembly_STARS["ϕ_flip"][main_mm_indices], missing=>NaN), 
    replace(assembly_DM["Merger_Map"][4,MMdm_indices] ./ assembly_DM["Merger_Map"][2,MMdm_indices], 0.0=>NaN), #yscale="log", 
    gridsize=bin_factor,
    extent=[ϕmin,ϕmax, ymin,ymax], 
    cmap="BuPu", zorder=1, norm=pltcolors.LogNorm() )

#ax.set_yticks([1e-2,1e-1,1e0,1e1,1e2,1e3,1e4,1e5,1e6])
ax.set_xlabel("Flip Angle ϕ [°]")
ax.set_ylabel("DM Mass Ratio")
ax.set_title("Most Massive Merger")
#ax.grid()
fig.colorbar(h, ax=ax)

scale=0.7

fig.set_size_inches(16scale, 9*scale)
fig.tight_layout()
show()

fig.savefig(joinpath(@__DIR__, "plots/M2mostmassive_vs_flipangles_MSTARSgt$(mass_ST_thr).png"), bbox_inches="tight", pad_inches=.1)


In [ ]:
# δj vs b-value

bin_factor  = 50
ϕmin        = 0
ϕmax        = 180
ymin        = -2
ymax        = 6
matplotlib  = pyimport("matplotlib")
axes_grid1  = pyimport("mpl_toolkits.axes_grid1")
pltcolors   = pyimport("matplotlib.colors")
matplotlib.use("qt5agg")




fig, ax = subplots()

h = ax.hexbin( replace(assembly_STARS["ϕ_flip"][main_mm_indices], missing=>NaN), 
    replace(assembly_DM["Merger_Map"][4,MMdm_indices] ./ assembly_DM["Merger_Map"][2,MMdm_indices], 0.0=>NaN), #yscale="log", 
    gridsize=bin_factor,
    #extent=[ϕmin,ϕmax, ymin,ymax], 
    cmap="BuPu", zorder=1, norm=pltcolors.LogNorm() )

#ax.set_yticks([1e-2,1e-1,1e0,1e1,1e2,1e3,1e4,1e5,1e6])
ax.set_xlabel("Flip Angle ϕ [°]")
ax.set_ylabel("log10( DM Mass Ratio )")
ax.set_title("Most Massive Merger")
#ax.grid()
fig.colorbar(h, ax=ax)

scale=0.7

fig.set_size_inches(16scale, 9*scale)
fig.tight_layout()
show()

#fig.savefig(joinpath(@__DIR__, "plots/_MSTARSgt$(mass_ST_thr).png"), bbox_inches="tight", pad_inches=.1)
